In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

# Beam Class Regression Tests
----

In [3]:
from distgen import Generator
input_file = "../examples/data/rad.gaussian.in.json"
gen = Generator(input=input_file,verbose=0)
beam = gen.beam()

# Beam weights must be normalized:

$\sum_i w_i = 1$

In [4]:
sum_weights = np.sum(beam['w'])
err = sum_weights-1
assert err.magnitude==0, 'Error: weights unnormalized, sum = !'+str(sum_weights)

In [8]:
from distgen.beam import Beam2
b = Beam2()

['x', 'px', 'y', 'py', 'z', 'pz', 't', 'w', 'r', 'theta', 'pr', 'ptheta', 'xp', 'yp', 'thetax', 'thetay', 'species']


In [6]:
b['x']=5

In [7]:
b['x']

5